In [1]:
import tensorflow as tf
import numpy as np
import cv2
import time

# 格式化成2016-03-20 11:45:39形式
print (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

2020-03-30 16:16:29


In [2]:
# 用户层面
# 传入阈值(nms的阈值，还有置信度的阈值)
# threshold=[0.5, 0.6, 0.7]
# nms_threshold = [0.7, 0.7, 0.7]
# weights_path = ['pnet_weights', 'rnet_weights', 'onet_weights']
# 是否只检测最大人脸 max_face = True
# 是否保存人脸 save_face = False
# 如果保存人脸，需要设置保存的位置 save_dirt = './'
# 格式化输出人脸检测到的时间 print_time = True
# 是否检测安全帽的佩戴 detect_hat = False
# 是否打印辅助信息 print_message = True
# 图片缩放方式  resize_type = 'with_loss/without_loss'
# 填充方式 padding_type = 'mask/nomask'
# factor: 缩放比例系数

# Returns:
# rects, landmark, ret
# [], [], 1 or 0


In [3]:
class MTCNN:
    """MTCNN，人脸检测，关键点定位
    
    作用：
        1.对输入的图片进行人脸检测，如果图片中
        存在人脸，则输出人脸框的预测位置
        2.输出人脸关键点的坐标，(left eyes),
        (right eyes), (nose), (left mouth),
        (right mouth)
    
    Args:
        threshold (list, float, optional): 包含三个浮点型数
            的列表，代表pnet,rnet,onet网络的置信度. 如果人脸检测效果
            不太好，可以适度调整该参数，检测多过多人脸，或者误将
            非人脸当作人脸，可以调大list中的三个数，同理，如果检
            测成功率较低，可以调低三个数，调整时，优先调整第一个
            数，其次是第二个，最后是第三个数.
        nms_threshold (list, float, optional): 包含三个浮点型数
            的列表，用以对三个网络的非极大值抑制设定阈值.一般不需要对
            该参数作出过多调整如果检测成功率较低，调整threshold参数，
            如果能够检测出人脸，不过人脸框的大小位置有问题可以调整该参数，
            一般情况下，数字调大，人脸框减少，数字调小，人脸框变多.
            Note: 调整此参数，可能会对检测速度产生一定影响.
        weight_paths (list, string, optional): pnet,rnet,onet
            网络预训练权重的文件位置，需要".h5"格式的文件。如果不提供
            该参数，则去默认路径("./weight_paths")寻找，找不到则报错
        max_face (bool, optional): 是否只检测最大人脸，True则只会找
            到图片中的最大人脸，False会尽可能找到图片中所有的人脸。
        save_face (bool, optional): 是否保存检测到的人脸区域，True
            则会保存检测提取出人脸区域，并且保存在本地，保存目录需要设置，
            False则不保存人脸。
        save_dirt (string optional): 如果设置了save_face=True,
            则一定需要设置该值，该变量代表保存人脸图片的位置。
        print_time (bool optional): 是否格式化输出检测到人脸的时间。
            True for yes. False for no.
        print_message (bool optional): 是否打印辅助信息。辅助信息
            包括程序现在进行的步骤，以及一些额外信息，实际运用时候，可
            以设置为false. 设置为false后只会打印基本信息。
        detect_hat (bool optional): 是否检测安全帽的佩戴，根据实际
            情况选择是否开启。开启后会检测人脸是否有佩戴安全帽，会打印
            相关信息。
        resize_type ('with_loss/without_loss', optional): 选择
            resize图像的方式。with_loss：直接缩放图片大小，会造成长
            方形图片扭曲失真。without_loss：以边缘填充的方式resize
            图片，图片会边大，不过保留了图片完整信息。
        resize_shape (tuple int): 用户输入的图片尺寸。
        padding_type ('mask/nomask', optional): 选择pnet网络获取
            图片的填充格式。
        factor (float, optional): pnet网络需要图片的缩放比例，默认为
            0.709,无特殊需求，一般不修改。修改可能会造成速度与准确度的降低
        
        
    Returns:
        [], [], bool
        rectangles (list, float): 返回检测到的人脸框的左上角坐标和右下角
            坐标。如果未检测到人脸框，返回空的list
        landmark (list, float): 返回检测到的左眼，右眼，鼻子，左嘴，右嘴
            的坐标。如果为检测到人脸，返回空的list
        have_face (bool): 如果检测到人脸，返回True, 否则返回False
        init_shape (tuple int optional): 图片的初始化大小

    Note: Requires Tensorflow v2.0 or later.  
    
    """
    
    def __init__(self,
                 threshold=[0.5, 0.6, 0.7],
                 nms_threshold=[0.7, 0.7, 0.7],
                 weight_paths=['./weight_path/pnet.h5',
                               './weight_path/rnet.h5',
                               './weight_path/onet.h5'],
                 max_face=False,
                 save_face=False,
                 save_dirt="./",
                 print_time=True,
                 print_message=True,
                 detect_hat=False,
                 resize_type='without_loss',
                 resize_shape=(80, 80),
                 padding_type='mask',
                 factor=0.709):
        """MTCNN初始化函数"""
        
        print("MTCNN TEXT VERSION:2.1 BY Luo-DH, NBUT")
        
        self.threshold = threshold # 三个网络的阈值
        self.nms_threshold = nms_threshold # nms的阈值
        self.weight_paths = weight_paths # 权重保存的路径
        self.max_face = max_face # 是否只检测最大人脸
        self.save_face = save_face # 是否保存检查到的人脸
        self.save_dirt = save_dirt # 设置保存图像的路径
        self.print_time = print_time # 是否打印检测到人脸的时间
        self.print_message = print_message # 是否打印debug信息
        self.detect_hat = detect_hat # 是否检测安全帽
        self.resize_type = resize_type # resize图像的方式
        self.resize_shape = resize_shape # 初始化缩放的尺寸
        self.padding_type = padding_type # 填充方式
        self.__factor = factor # pnet图像的缩放比例
        
        # 获得一系列的缩放系数
        self.__scales = self.__get_scales()
        
        self.__net = Net(self.print_message,
                         self.resize_shape)
        
        self.__pnet = Pnet(self.weight_paths[0],
                           self.__scales,
                           self.threshold[0],
                           self.nms_threshold[0])
        
        self.__rnet = Rnet(self.weight_paths[1],
                           self.threshold[1],
                           self.nms_threshold[1])

        self.__onet = Onet(self.weight_paths[2],
                           self.threshold[2],
                           self.nms_threshold[2])
        
    ########################################
    # 获得一系列的比例系数
    ########################################
    def __get_scales(self):
        """这个函数用于获得缩放比例
        
        将原始图片进行缩放，保存缩放系数，保证缩小成最小值
        后，长和宽仍然大于12，否则无法传入pnet网络
        
        Args:
            shape: 输入图片的长和宽 (int, int)
            
        Returns:
            缩放比例，list. 
            Fof example: []
        
        """
        
        i = 0
        
        scales = []

        while True:

            scale = self.__factor ** i

            tmp_width = self.resize_shape[0] * scale
            tmp_height = self.resize_shape[1] * scale

            # 如果缩放成小于12，则不符合要求
            if min(tmp_width, tmp_height) <= 13:

                break

            scales.append(scale) # 符合要求的值放入__scale中
            i += 1 # i的值每次加一，以便减小scale的值
            
        return scales
        
    ########################################
    # 构造打印debug信息的函数以及装饰器，
    # 如果print_message==False，则不打印信息
    #######################################
    def can_show_message(func):
        def inner(self, mess):
            if self.print_message == False:
                pass
            else:
                return func(self, mess)
        return inner
        
    @can_show_message
    def print_messages(self, mess):
        print(mess)
        print("*"*10)

    ######################################
    # 保存图片信息
    ######################################
    def get_message(self, image):
        width = image.shape[0]
        height = image.shape[1]

        big_side = width if width>height else height
        
        self.width_scale = big_side / self.resize_shape[0]
        self.height_scale = big_side / self.resize_shape[1]
        
        
        
    ######################################
    # 主函数，对用户检测人脸提供接口
    ######################################
    def detect_face(self, image):
        """该函数用于人脸检测，对用户主要接口
        
        用户只需要调用该方法，并且传入需要检测人脸的
        图片，需要类型为RGB的三通道图片(opencv读取
        的图片需要作通道转换),并且图片的最小尺寸要大
        于12x12
        
        Args:
            image(dtype=np.ndarray): 用户传入的
                待检测人脸的图片。   
                    shape = (x, x, 3) RGB顺序
        
        Returns:
            [], [] , bool 返回矩形框坐标和五官坐标
            如果没有检测到人脸，则返回两个空list和False
        
        """
        
        # 预处理，为最后还原信息使用
        self.get_message(image)
        
        # pnet网络处理
        rects, ret = self.pnet_processing(image.copy())
        
        if ret == False:
            return [], [], False
        
        # rnet网络处理
        rects, ret = self.rnet_processiong(rects, image.copy())
  
        if ret == False:
            return [], [], False
        
        # onet网络处理
        rects, landmark, ret = self.onet_processing(rects, 
                                               image.copy())
        
        if ret == False:
            return [], [], False
        
        # 边界框最后调整
        self.fix_rects(rects)
        
        # 如果保存图片
        self.to_save_face(rects, image.copy(), self.save_dirt) 
        
        # 如果需要获取安全帽
        self.to_get_hat(rects, image.copy())

        return rects, landmark, ret
        
    ######################################
    # 将得到的边界还原到原图合适的比例
    ######################################
    def fix_rects(self, rects):
        
        for rect in rects:

            width = rect[2] - rect[0]
            height = rect[3] - rect[1]

            rect[0] = rect[0] * self.width_scale
            rect[1] = rect[1] * self.height_scale
            rect[2] = rect[0] + width * self.width_scale
            rect[3] = rect[1] + height * self.height_scale
    
            
    ######################################
    # 根据需要截取图片，保存到本地
    ######################################
    def need_to_save_face(func):
        def inner(self, rects, image, dirt):
            
            if self.save_face == False:
                pass
            else:
                return func(self, rects, image, dirt)
        return inner
    
    @need_to_save_face
    def to_save_face(self, rects, image, dirt):
        
        name = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + str(time.clock())
        
        if self.max_face == True:
            
            img_ = self.to_get_max_face(rects, image)
            
            if(cv2.imwrite(dirt+"/"+name+".jpg", img_)):
                    self.print_messages("成功保存人脸到{}".format(dirt+"/"+name+".jpg"))
            
            
        else:
            
            for i in range(len(rects)):
            
                # 获取每个矩形框
                rect = rects[i]

                img_ = self.to_get_all_faces(rect, image)
                #show_img(img_)
                if(cv2.imwrite(dirt+"/"+name+".jpg", img_)):
                    self.print_messages("成功保存人脸到{}".format(dirt+"/"+name+".jpg"))
    
    
    def to_get_all_faces(self, rect, image):
        """获得图像中的所有人脸"""
        img_ = image.copy()[int(rect[1]): int(rect[3]),
                            int(rect[0]): int(rect[2])]
        
        return img_
    
    def to_get_max_face(self, rects, image):
        """获取图像中的最大人脸"""
        areas = []
        for rect in rects:
            width = rect[2] - rect[0]
            height = rect[3] - rect[1]
            
            area = width*height
            
            areas.append(area)
            
        index = np.argmax(np.array(areas), axis=0) #竖着比较，返回行号
        
        img_ = self.to_get_all_faces(rects[index], image)
        
        return img_
    
    ######################################
    # 获取安全帽
    ######################################
    
    def need_to_detect_hat(func):
        def inner(self, rects, image):
            
            if self.detect_hat == False:
                pass
            else:
                return func(self, rects, image)
        return inner
    
    @need_to_detect_hat
    def to_get_hat(self, rects, image):
        
        for rect in rects:
            
            height = rect[3] - rect[1]
            
            rect[1] = rect[3] - 1.5*height
            
        rects = self.__net._rect2square(rects)
        
        rects = self.__net._trimming_frame(rects,
                                   width = image.shape[0],
                                   height = image.shape[1])   
        print(rects[0][3]-rects[0][1])
        print(rects[0][2]-rects[0][0])
        # 保存带有安全帽的人脸路径
        dirt = self.save_dirt + "/hat"
        
        self.to_save_face(rects, image, dirt)
        

        
        
    ######################################
    # 定义pnet网络需要的一系列处理函数以及过程
    # 1. 对图片进行归一化
    #     1).按照指定尺寸缩放
    #     2).对缩放后的图片转换数据类型并且归一化
    # 2. 获得pnet的输入
    #     1).按照比例系数进行缩放
    # 3. 经过pnet网络，获得预测人脸框
    ######################################
    def pnet_processing(self, image):
        """封装pnet需要的函数"""
        
        # 传入图片，归一化
        image_ = self.__norm(image.copy())
        
        # 获得pnet的输入
        pnet_need_imgs = self.__get_pnet_need_imgs(image_)
        
        # 经过pnet网络，得到pnet预测的人脸框
        pnet_got_rects = self.__pnet.forward(pnet_need_imgs)
        
        if len(pnet_got_rects) == 0:
            
            self.print_messages("准备检测下一张图片")
            
            return [], False
        
        else:
            
            self.print_messages("pnet一共获得{}个预测人脸框".format(len(pnet_got_rects)))

        return pnet_got_rects, True
    
    def __norm(self, image):
        """对输入的图片作归一化"""
        
        # 颜色通道转换
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 对图片按用户指定的尺寸缩放
        if self.resize_type == 'with_loss':
            image = self.image_resize(image, 
                                      self.resize_shape)
        else:
            image = self.image_resize_padding(image, 
                                              self.resize_shape)
        
        # 对图片作归一化处理
        #image = tf.cast(image, tf.float32)
        image = (image - 127.5) / 127.5

        return image
        

    def image_resize(self,
                     image,
                     size):
        """图像缩放"""
        image = cv2.resize(image.copy(),
                           (size[0], size[1]))

        return image
  

    def image_resize_padding(self, 
                     image, 
                     size):
        """缩放函数
        
        Args:
            image: 待缩放的原始图像
            size: (tuple, (int, int)),缩放后的尺寸
        
        """
        
        width = image.shape[0] # 获得图像的宽
        height = image.shape[1]# 获得图像的高

        # 选择大的边作为resize后的边长
        side_length = image.shape[0] if width \
                                      > height \
                                     else height

        mask = self.mask_template((side_length, side_length))

        mask[0:width, 0:height] = image # 获取padding后的图像
        
        image = self.image_resize(mask, size)

        return image

        
    def mask_template(self, shape):
        """图片掩码模板
        
        根据用户输入resize图片的尺寸，
        制作模板，方便获取不同大小的pnet
        图片的需求
        
        """
        sss = np.zeros([shape[0], shape[1]],dtype=np.uint8)
        sss = cv2.cvtColor(sss, cv2.COLOR_GRAY2RGB)
        sss = (sss-127.5)/127.5 
        
        return sss
    
    ##############################################
    # 按照比例系数获得pnet的输入
    ##############################################
    def __get_pnet_need_imgs(self, image):
        """获得pnet输入需要的一系列图片
        
        通过scales对原始图片进行缩放，被
        缩放的图片填充回原图大小，打包返回
        
        Args:
            image: 需要被检测人脸的图像
            
        Returns：
            np.array((image1, image2, image3 ...))
            shape = (n, x, x, 3, np.array)
        
        """
        
        image_width = image.shape[0]
        image_height = image.shape[1]
        image_list = []
        
        for scale in self.__scales:
            
            sss_ = self.mask_template(self.resize_shape)
        
            width = int(scale*image_width)
            height = int(scale*image_height)
            size = (width, height)
            img_tmp = self.image_resize(image.copy(), size) 
                                        
            
            sss_[0:width, 0:height] = img_tmp

            image_list.append(sss_)
            
            #how_img(sss_)
            
        return np.array(image_list)
    
    ##########################################
    # rnet网络处理
    # 1. 归一化原始图像
    # 2. 获得rnet网络的输入
    # 3. 经过网络得到输出
    ##########################################
    def rnet_processiong(self, rects, image):
        """封装rnet需要的函数"""
        
        # 传入图片，归一化
        image_ = self.__norm(image.copy())
        
        # 获取rnet需要的输入数据
        rnet_need_imgs = self.get_rnet_need_imgs(image_, 
                                                 rects)
        
        # 经过网络获得预测结果
        rnet_got_rects = self.__rnet.forward(rnet_need_imgs, 
                                             rects)
        
        if len(rnet_got_rects) == 0:
            
            self.print_messages("准备检测下一张图片")
            
            return [], False
        
        else:
            
            self.print_messages("rnet一共获得{}个预测人脸框".format(len(rnet_got_rects)))

        return rnet_got_rects, True        
    
    
    def get_rnet_need_imgs(self, image, rects):
        """获得一系列rnet需要的格式的图片

        Args:
            rects: 经过pnet处理得到的所有矩形框

        """

        rnet_need_imgs = self.__get_net_need_imgs(rects, 
                                                  image, 
                                                  'rnet')

        return np.array(rnet_need_imgs)


    def __get_net_need_imgs(self, rects, image, net_type='rnet'):
        """获取输入网络图像的通用方法

        判断rects的长度，如果为0，则返回空list

        rects的x1，y1不能大于图像长或者宽

        Args：
            image: 归一化后的原始图像
            rects: 经过网络处理后得到的矩形框
            net_type: 网络的类型，pnet或者rnet，(string)

        """

        need_imgs = []
        
        for rect in rects:

            tmp_roi = image.copy()[int(rect[1]): int(rect[3]), \
                                int(rect[0]): int(rect[2])]

            if net_type == 'rnet':

                tmp_roi = cv2.resize(tmp_roi, (24, 24))

                need_imgs.append(tmp_roi)

            elif net_type == 'onet':

                tmp_roi = cv2.resize(tmp_roi, (48, 48))

                need_imgs.append(tmp_roi)

        return need_imgs
    
    ##########################################
    # onet网络处理
    # 1. 归一化原始图像
    # 2. 获得onet网络的输入
    # 3. 经过网络得到输出
    ##########################################
    def onet_processing(self, rects, image):
        """封装rnet需要的函数"""
        
        # 传入图片，归一化
        image_ = self.__norm(image.copy())
        
        # 获取onet需要的输入数据
        onet_need_imgs = self.get_onet_need_imgs(image_, 
                                                 rects)
        
        # 经过网络获得预测结果
        onet_got_rects, landmarks = self.__onet.forward(onet_need_imgs, 
                                             rects)
        
        if len(onet_got_rects) == 0:
            
            self.print_messages("准备检测下一张图片")
            
            return [], [], False
        
        else:
            
            self.print_messages("onet一共获得{}个预测人脸框".format(len(onet_got_rects)))

        return onet_got_rects, landmarks, True 
    
    ##########################################
    # 获取Onet输入的数据
    ##########################################


    def get_onet_need_imgs(self, image, rects):
        """获得一系列onet需要的格式的图片

        Args:
            rects: 经过rnet处理得到的所有矩形框

        """

        onet_need_imgs = self.__get_net_need_imgs(rects,
                                                  image,
                                                  'onet')

        return np.array(onet_need_imgs)

In [4]:
class Net:
    
    def __init__(self, 
                 print_message=True,
                 resize_shape=(80, 80)):
        """初始化"""
        self.print_message = print_message
        self.width = resize_shape[0]
        self.height = resize_shape[1]
    
    
    ########################################
    # 构造打印debug信息的函数以及装饰器，
    # 如果print_message==False，则不打印信息
    ########################################
    def can_show_message(func):
        def inner(self, mess):
            if self.print_message == False:
                pass
            else:
                return func(self, mess)
        return inner
        
    @can_show_message
    def print_messages(self, mess):
        print(mess)
        print("*"*10)
        
        
    #######################################
    # 非极大值抑制
    #######################################
    @staticmethod
    def _nms(rectangles, threshold):
        """非极大值抑制

        经过网络得到的矩形框可能有较多的重叠，我们先取出
        概率最大的框，与其余所有框分别作交并比(IOU), IOU
        在设定阈值以内的框则去除。

        经过nms处理，可以去除重合度相对高的矩形框，保留得
        分概率比较高，并且重合度相对低的矩形框

        Args:
            rectangles: 所有矩形框的坐标值以及得分 (numpy.ndarray)
                For example：
                 [x1,     y1,    x2,    y2,     score]
                [[132.    17.   138.    23.     0.83198518]
                 [133.    17.   139.    23.     0.5036146 ]
                 [138.    20.   144.    26.     0.533454  ]
                 [117.    28.   123.    34.     0.50165093]]

            threshold: 阈值 (float)

        Returns:
            经过nms处理后剩余的矩形框的坐标值以及概率得分(numpy.ndarray)
                For example：
                 [x1,     y1,    x2,    y2,     score]
                [[132.    17.   138.    23.     0.83198518]
                 [117.    28.   123.    34.     0.50165093]]

        """
        if len(rectangles)==0:
            return rectangles
        boxes = np.array(rectangles)
        x1 = boxes[:, 0]
        y1 = boxes[:, 1]
        x2 = boxes[:, 2]
        y2 = boxes[:, 3]
        s  = boxes[:, 4]
        area = np.multiply(x2-x1+1, y2-y1+1)
        I = np.array(s.argsort())
        pick = []
        while len(I)>0:
            xx1 = np.maximum(x1[I[-1]], x1[I[0:-1]])
            yy1 = np.maximum(y1[I[-1]], y1[I[0:-1]])
            xx2 = np.minimum(x2[I[-1]], x2[I[0:-1]])
            yy2 = np.minimum(y2[I[-1]], y2[I[0:-1]])
            w = np.maximum(0.0, xx2 - xx1 + 1)
            h = np.maximum(0.0, yy2 - yy1 + 1)
            inter = w * h
            o = inter / (area[I[-1]] + area[I[0:-1]] - inter)
            pick.append(I[-1])
            I = I[np.where(o<=threshold)[0]]
        result_rectangle = boxes[pick].tolist()
        return result_rectangle

    
    #######################################
    # 将矩形框修整为正方形
    #######################################
    @staticmethod
    def _rect2square(rectangles):
        """将矩形框修整为正方形

        经过pnet或者rnet得到的矩形框大多为长方形，但是rnet或者onet
        网络的输入格式是正方形，因此我们取长方形较长的边，作为矫正后
        正方形的边，得到的正方形区域会比原来的矩形框稍大。

        Args:
            rectangles: 矩形框的坐标值
                For example：
                 [x1,     y1,    x2,    y2,     score]
                [[132.    17.   138.    23.     0.83198518]
                 [117.    28.   123.    34.     0.50165093]]

        Returns:
            与输入的类型以及大小都相同

        """
        rectangles = np.array(rectangles)
        w = rectangles[:,2] - rectangles[:,0]
        h = rectangles[:,3] - rectangles[:,1]
        l = np.maximum(w,h).T
        rectangles[:,0] = rectangles[:,0] + w*0.5 - l*0.5
        rectangles[:,1] = rectangles[:,1] + h*0.5 - l*0.5
        rectangles[:,2:4] = rectangles[:,0:2] + np.repeat([l], 2, axis = 0).T
        return rectangles

    
    #######################################
    # 对矩形框进行调整，避免出现数值不合理的情况
    #######################################
    def _trimming_frame(self, rectangles, width, height):
        """对矩形框进行调整，避免出现数值不合理的情况

        无论经过网络得到的矩形框还是其他处理后得到的矩形框，
        值有可能不合理(例如小于零或者大于原图像的长宽)，此
        方法的目的就是使矩形框不超过合理范围

        Args:
            rectangles: 矩形框的坐标值
                For example：
                 [x1,     y1,    x2,    y2,     score]
                [[132.    17.   138.    23.     0.83198518]
                 [117.    28.   123.    34.     0.50165093]]

        Returns:
            与输入的类型以及大小都相同

        """
        for j in range(len(rectangles)):

            rectangles[j][0] = max(0, int(rectangles[j][0]))
            rectangles[j][1] = max(0, int(rectangles[j][1]))
            rectangles[j][2] = min(width, int(rectangles[j][2]))
            rectangles[j][3] = min(height, int(rectangles[j][3]))
            
            if rectangles[j][0] >= rectangles[j][2]:
                rectangles[j][0] = 0
            elif rectangles[j][1] > rectangles[j][3]:
                rectangles[j][1] = 0

        return rectangles

In [5]:
class Pnet(Net):
    """Pnet网络前向传播"""
    
    def __init__(self, 
                 weight_path, 
                 scales,
                 threshold,
                 nms_threshold):
        """初始化，构建网络架构，读取网络权重"""
        
        super().__init__()
        
        self.__threshold = threshold
        
        self.__nms_threshold = nms_threshold
        
        self.__model = self.__create_model()
        
        self.__model.load_weights(weight_path, by_name=True)
        
        self.__scales = scales
        
        
    #######################################
    # 主函数，前向传播函数，获得并且处理pnet得到的结果
    # 1.获取网络预测值
    # 2.处理预测结果
    #    1).找到大于阈值的位置
    #    2).找到边框的位置
    #    3).获取偏移量
    #    4).获取得分值
    #######################################
    def forward(self, pnet_need_imgs):
        """前向传播函数，主要接口
        
        这个函数将完成pnet的全过程，输入图片
        获得预测值，并对预测值进行处理
        
        Args:
            pnet_need_imgs:(np.array), shape=(x, h, w, 3)
            
        Returns:
            返回处理得到的预测人脸框. (np.array)
        
        """
        self.width = pnet_need_imgs[0].shape[0]
        self.height = pnet_need_imgs[0].shape[1]
        
        rectangles = []
        
        # 传入网络
        out = self.__model.predict(pnet_need_imgs)
        
        # 获取矩形框
        boundingbox = self.__get_boundingbox(out)
   
        if len(boundingbox) == 0:
            
            self.print_messages("该张图像在pnet网络检测不到人脸")
            
            return []
        
        # 将矩形框调整成正方形
        boundingbox = self._rect2square(boundingbox)

        # 避免数值不合理
        boundingbox = self._trimming_frame(boundingbox,
                                           width = self.width,
                                           height = self.height)

        # nms
        boundingbox = self._nms(boundingbox, 0.3)
        
        self.print_messages("Pnet网络处理完毕")

        return self._nms(boundingbox, self.__nms_threshold)
    
    #######################################
    # 找到边框位置
    #######################################
    def __get_boundingbox(self, out):
        """这个方法主要用于判断大于阈值的坐标，并且转换成矩形框
        
        Args:
            cls_prob: pnet网络输出得到的第一个array
        
        """
        
        boundingbox = []
        
        #scores = []
        
        for i in range(len(self.__scales)):
            
            scale = self.__scales[i]
            
            cls_prob = out[0][i, :, :, 1]
            
            (x, y), bbx = self.__boundingbox(cls_prob, scale)

            if bbx.shape[0] == 0:
                continue
                
            scores= np.array(out[0][i, x, y, 1][np.newaxis, :].T)
            
            offset = out[1][i, x, y]*12*(1/scale) 
            
            bbx = bbx + offset
            bbx = np.concatenate((bbx, scores), axis=1)
            
#             # 将矩形框调整成正方形
#             bbx = self._rect2square(bbx)

#             # 避免数值不合理
#             bbx = self._trimming_frame(bbx)

#             # nms
#             bbx = self._nms(bbx, 0.3)            
            
            
            for b in bbx:
                
                boundingbox.append(b)
            
        return np.array(boundingbox)
    
    def __boundingbox(self, cls_prob, scale):
        
        x, y = np.where(cls_prob > self.__threshold)

        bbx = np.array((y, x)).T

        left_top = np.fix(((bbx * 2) + 0) * (1/scale))
        
        right_down = np.fix(((bbx * 2) + 11) * (1/scale))

        return (x, y), np.concatenate((left_top, right_down), axis=1)
  
    #######################################
    # 定义网络架构
    #######################################
    @classmethod
    def __create_model(cls):
        """定义PNet网络的架构"""

        input = tf.keras.Input(shape=[None, None, 3])
        x = tf.keras.layers.Conv2D(10, (3, 3),
                                   strides=1,
                                   padding='valid',
                                   name='conv1')(input)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2],
                                  name='PReLU1')(x)
        x = tf.keras.layers.MaxPooling2D()(x)
        x = tf.keras.layers.Conv2D(16, (3, 3),
                                   strides=1,
                                   padding='valid',
                                   name='conv2')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2],
                                  name='PReLU2')(x)
        x = tf.keras.layers.Conv2D(32, (3, 3),
                                   strides=1, padding='valid', name='conv3')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2],
                                  name='PReLU3')(x)

        classifier = tf.keras.layers.Conv2D(2, (1, 1),
                                            activation='softmax',
                                            name='conv4-1')(x)
        bbox_regress = tf.keras.layers.Conv2D(4, (1, 1),
                                              name='conv4-2')(x)

        model = tf.keras.models.Model([input], [classifier, bbox_regress])

        return model


In [6]:
class Rnet(Net):
        
    def __init__(self, 
                 weight_path, 
                 threshold,
                 nms_threshold):
        """初始化，构建网络架构，读取网络权重"""
        
        super().__init__()
        
        self.__threshold = threshold
        
        self.__nms_threshold = nms_threshold
        
        self.__model = self.__create_model()
        
        self.__model.load_weights(weight_path, by_name=True)
       

    def __get_boundingbox(self, outs, pnet_got_rects):
        """这个函数用于得到加上偏移后的矩形框坐标

        Args:
            outs: 经过网络后得到的结果

        Attributes:

            offset: 经过网络后得到的偏移量. For example:

                [[[[ 0.0463337   0.01343044 -0.125744   -0.03012199]]

                  [[-0.04171417 -0.19884819  0.18158348  0.25635445]]]]

            x: 符合阈值条件的坐标值 For example:

                array([ 3, 11]),

        Returns: 加上偏移后的坐标值

            [[112.92667393  20.26860878 129.48512     39.39756013]

             [  6.28743928   0.30251127 115.16092964 129.81554615]]

        """

        # 人脸概率
        classifier = outs[0]

        # 偏移量
        offset = outs[1]

        x = np.where(classifier[:, 1] > self.__threshold)

        # 获得相应位置的offset值
        offset = offset[x, None]

        dx1 = np.array(offset[0])[:, :, 0]
        dy1 = np.array(offset[0])[:, :, 1]
        dx2 = np.array(offset[0])[:, :, 2]
        dy2 = np.array(offset[0])[:, :, 3]

        pnet_got_rects = np.array(pnet_got_rects)

        x1 = np.array(pnet_got_rects[x][:, 0])[np.newaxis, :].T
        y1 = np.array(pnet_got_rects[x][:, 1])[np.newaxis, :].T
        x2 = np.array(pnet_got_rects[x][:, 2])[np.newaxis, :].T
        y2 = np.array(pnet_got_rects[x][:, 3])[np.newaxis, :].T

        w = x2 - x1
        h = y2 - y1

        new_x1 = np.fix(x1 + dx1*w)
        new_x2 = np.fix(x2 + dx2*w)
        new_y1 = np.fix(y1 + dy1*h)
        new_y2 = np.fix(y2 + dy2*h)

        score = np.array(classifier[x, 1]).T


        boundingbox = np.concatenate((new_x1, 
                                      new_y1, 
                                      new_x2, 
                                      new_y2, 
                                      score), axis=1)

        return boundingbox
        

    def forward(self, rnet_need_imgs, pnet_got_rects):
        """核心函数，前向网络传播

        将输入数据传入网络中，并处理得到的结果

        Args:
            rects: 将pnet网络得到的预测人脸框提取出来，
                resize成24x24，作为rnet的输入

        Returns:
            左上角与右下角的坐标值(x, y). For example:

                [[125.0, 272.0, 142.0, 289.0, 0.9670179486274719],
                [123.0, 269.0, 145.0, 291.0, 0.9095805287361145],
                [79.0, 305.0, 91.0, 317.0, 0.8930739760398865]]

        """

        self.print_messages("开始通过Rnet网络进行处理")

        rnet_need_imgs = np.array(rnet_need_imgs)

        outs = self.__model.predict(rnet_need_imgs)

        boundingbox = self.__get_boundingbox(outs, pnet_got_rects)

        # 将矩形框变成正方形
        boundingbox = self._rect2square(boundingbox)

        # 微调，避免数值不合理
        boundingbox = self._trimming_frame(boundingbox, 
                                           image.shape[0],
                                           image.shape[1])

        # nms
        boundingbox = self._nms(boundingbox, self.__nms_threshold)
        
        return boundingbox

    @classmethod
    def __create_model(cls):
        """定义RNet网络的架构"""

        input = tf.keras.Input(shape=[24, 24, 3])
        x = tf.keras.layers.Conv2D(28, (3, 3), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv1')(input)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2], 
                                  name='prelu1')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=3, 
                                         strides=2, 
                                         padding='same')(x)

        x = tf.keras.layers.Conv2D(48, (3, 3), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv2')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2], 
                                  name='prelu2')(x)
        x = tf.keras.layers.MaxPooling2D(pool_size=3, 
                                         strides=2)(x)

        x = tf.keras.layers.Conv2D(64, (2, 2), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv3')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1, 2], 
                                  name='prelu3')(x)

        x = tf.keras.layers.Permute((3, 2, 1))(x)
        x = tf.keras.layers.Flatten()(x)

        x = tf.keras.layers.Dense(128, name='conv4')(x)
        x = tf.keras.layers.PReLU(name='prelu4')(x)

        classifier = tf.keras.layers.Dense(2, 
                                           activation='softmax', 
                                           name='conv5-1')(x)
        bbox_regress = tf.keras.layers.Dense(4, name='conv5-2')(x)

        model = tf.keras.models.Model([input], [classifier, bbox_regress])

        return model

In [7]:
class Onet(Net):
    
    def __init__(self, 
                 weight_path, 
                 threshold,
                 nms_threshold):
        """初始化，构建网络架构，读取网络权重"""
        
        super().__init__()
        
        self.__threshold = threshold
        
        self.__nms_threshold = nms_threshold
        
        self.__model = self.__create_model()
        
        self.__model.load_weights(weight_path, by_name=True)


    def __get_landmark(self, outs, rnet_got_rects):

        classifier = outs[0]
        x = np.where(classifier[:, 1] > self.__threshold)

        onet_pts = outs[2]

        offset_x1 = onet_pts[x, 0]
        offset_y1 = onet_pts[x, 5]
        offset_x2 = onet_pts[x, 1]
        offset_y2 = onet_pts[x, 6]
        offset_x3 = onet_pts[x, 2]
        offset_y3 = onet_pts[x, 7]
        offset_x4 = onet_pts[x, 3]
        offset_y4 = onet_pts[x, 8]
        offset_x5 = onet_pts[x, 4]
        offset_y5 = onet_pts[x, 9]

        x1 = rnet_got_rects[0][0]
        y1 = rnet_got_rects[0][1]
        x2 = rnet_got_rects[0][2]
        y2 = rnet_got_rects[0][3]

        w = x2 - x1
        h = y2 - y1

        onet_pts_x1 = np.array(offset_x1*w + x1)
        onet_pts_x2 = np.array(offset_x2*w + x1)
        onet_pts_x3 = np.array(offset_x3*w + x1)
        onet_pts_x4 = np.array(offset_x4*w + x1)
        onet_pts_x5 = np.array(offset_x5*w + x1)
        onet_pts_y1 = np.array(offset_y1*h + y1)
        onet_pts_y2 = np.array(offset_y2*h + y1)
        onet_pts_y3 = np.array(offset_y3*h + y1)
        onet_pts_y4 = np.array(offset_y4*h + y1)
        onet_pts_y5 = np.array(offset_y5*h + y1)

        onet_left_eye = np.concatenate((onet_pts_x1, 
                                        onet_pts_y1), axis=1)
        onet_right_eye = np.concatenate((onet_pts_x2, 
                                         onet_pts_y2), axis=1)
        onet_nose = np.concatenate((onet_pts_x3, 
                                    onet_pts_y3), axis=1)
        onet_left_mouth = np.concatenate((onet_pts_x4, 
                                          onet_pts_y4), axis=1)
        onet_right_mouth = np.concatenate((onet_pts_x5, 
                                           onet_pts_y5), axis=1)

        return (onet_left_eye, 
                onet_right_eye, 
                onet_nose, 
                onet_left_mouth, 
                onet_right_mouth)

    def __get_boundingbox(self, outs, rnet_got_rects):
        """这个函数用于得到加上偏移后的矩形框坐标

        Args:
            outs: 经过网络后得到的结果

        Attributes:

            offset: 经过网络后得到的偏移量. For example:

                [[[[ 0.0463337   0.01343044 -0.125744   -0.03012199]]

                  [[-0.04171417 -0.19884819  0.18158348  0.25635445]]]]

            x: 符合阈值条件的坐标值 For example:

                array([ 3, 11]),

        Returns: 加上偏移后的坐标值

            [[112.92667393  20.26860878 129.48512     39.39756013]

             [  6.28743928   0.30251127 115.16092964 129.81554615]]

        """

        # 人脸概率
        classifier = outs[0]

        # 偏移量
        offset = outs[1]

        x = np.where(classifier[:, 1] > self.__threshold)

        # 获得相应位置的offset值
        offset = offset[x, None]

        dx1 = np.array(offset[0])[:, :, 0]
        dy1 = np.array(offset[0])[:, :, 1]
        dx2 = np.array(offset[0])[:, :, 2]
        dy2 = np.array(offset[0])[:, :, 3]

        rnet_got_rects = np.array(rnet_got_rects)

        x1 = np.array(rnet_got_rects[x][:, 0])[np.newaxis, :].T
        y1 = np.array(rnet_got_rects[x][:, 1])[np.newaxis, :].T
        x2 = np.array(rnet_got_rects[x][:, 2])[np.newaxis, :].T
        y2 = np.array(rnet_got_rects[x][:, 3])[np.newaxis, :].T

        w = x2 - x1
        h = y2 - y1

        new_x1 = np.fix(x1 + dx1*w)
        new_x2 = np.fix(x2 + dx2*w)
        new_y1 = np.fix(y1 + dy1*h)
        new_y2 = np.fix(y2 + dy2*h)

        score = np.array(classifier[x, 1]).T


        boundingbox = np.concatenate((new_x1, 
                                      new_y1, 
                                      new_x2, 
                                      new_y2, 
                                      score), axis=1)

        return boundingbox
    
    
    def forward(self, onet_need_imgs, rnet_got_rects):
        """核心函数，前向网络传播

        将输入数据传入网络中，得到预测人脸框的坐标值以及
        五官的五个坐标值，以数组形式返回，方便用户使用

        Returns:
            左上角与右下角的坐标值(x, y). For example:

            array([[134.,  19., 138.,  23.],
                   [ 34.,  86.,  64., 116.],
                   [ 11.,  18.,  99., 107.]])

        """

        self.print_messages("开始通过Rnet网络进行处理")

        outs = self.__model.predict(onet_need_imgs)

        boundingbox = self.__get_boundingbox(outs, rnet_got_rects)

        boundingbox = self._rect2square(boundingbox)

        boundingbox = self._trimming_frame(boundingbox,
                                           width = image.shape[0],
                                           height = image.shape[1])

        landmark = self.__get_landmark(outs, rnet_got_rects)

        return boundingbox, landmark
        
    @classmethod
    def __create_model(cls):
        """定义ONet网络的架构"""

        input = tf.keras.layers.Input(shape = [48,48,3])
        # 48,48,3 -> 23,23,32
        x = tf.keras.layers.Conv2D(32, (3, 3),
                                   strides=1, 
                                   padding='valid', 
                                   name='conv1')(input)
        x = tf.keras.layers.PReLU(shared_axes=[1,2],
                                  name='prelu1')(x)
        x = tf.keras.layers.MaxPool2D(pool_size=3, 
                                      strides=2, 
                                      padding='same')(x)
        # 23,23,32 -> 10,10,64
        x = tf.keras.layers.Conv2D(64, (3, 3), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv2')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1,2],
                                  name='prelu2')(x)
        x = tf.keras.layers.MaxPool2D(pool_size=3, 
                                      strides=2)(x)
        # 8,8,64 -> 4,4,64
        x = tf.keras.layers.Conv2D(64, (3, 3), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv3')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1,2],
                                  name='prelu3')(x)
        x = tf.keras.layers.MaxPool2D(pool_size=2)(x)
        # 4,4,64 -> 3,3,128
        x = tf.keras.layers.Conv2D(128, (2, 2), 
                                   strides=1, 
                                   padding='valid', 
                                   name='conv4')(x)
        x = tf.keras.layers.PReLU(shared_axes=[1,2],
                                  name='prelu4')(x)
        # 3,3,128 -> 128,12,12
        x = tf.keras.layers.Permute((3,2,1))(x)

        # 1152 -> 256
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(256, name='conv5') (x)
        x = tf.keras.layers.PReLU(name='prelu5')(x)

        # 鉴别
        # 256 -> 2 256 -> 4 256 -> 10
        classifier = tf.keras.layers.Dense(2, 
                                           activation='softmax',
                                           name='conv6-1')(x)
        bbox_regress = tf.keras.layers.Dense(4,name='conv6-2')(x)
        landmark_regress = tf.keras.layers.Dense(10,name='conv6-3')(x)

        model = tf.keras.models.Model([input], [classifier, bbox_regress, landmark_regress])

        return model

In [8]:
def show_img(image):
    cv2.imshow("image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
mtcnn = MTCNN(threshold=[0.5, 0.6, 0.7],
              nms_threshold=[0.7, 0.7, 0.7],
              weight_paths=['./weight_path/pnet.h5',
                            './weight_path/rnet.h5',
                            './weight_path/onet.h5'],
              max_face=False,
              save_face=True,
              save_dirt="./output",
              print_time=True,
              print_message=True,
              detect_hat=True,
              resize_type='without_loss',
              padding_type='mask'
              )

MTCNN TEXT VERSION:2.1 BY Luo-DH, NBUT


In [10]:
image = cv2.imread("/home/luo/桌面/hat11.jpg")
# img_ = cv2.copyMakeBorder(image,
#                           0,0,0,250, 
#                           cv2.BORDER_CONSTANT,
#                           value=[0,255,0])
# print(img_.shape)
rects, _, _ = mtcnn.detect_face(image)

Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:313.909255.jpg
**********
124.0
124.0
成功保存人脸到./output/hat/2020-03-30 16:16:313.911411.jpg
**********


In [11]:
# img_ = cv2.rectangle(image,
#                      (int(rects[0][0]), int(rects[0][1])), 
#                      (int(rects[0][2]),int(rects[0][3])),
#                      (0, 255, 0), 4)
# show_img(img_)

In [12]:
# dfgdf

In [13]:
import glob
data_list = glob.glob("./hat_data/*") # 5585张图片
print(len(data_list))
i = 0
cost_time = 0
for path in data_list:
    
    img = cv2.imread(path)
    
    #img = cv2.resize(img, (80, 80))

    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #img = (img-127.5)/127.5
    
    start = time.clock()
    _, _, ret = mtcnn.detect_face(img)
    end = time.clock()
    cost_time += ((end-start)*1000)
    
    if ret==True:
        
        i += 1
        
        
print("准确率:{}".format(i/len(data_list)))
print("平均耗时:{}".format(cost_time/len(data_list)))

895
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:314.186509.jpg
**********
76.0
86.0
成功保存人脸到./output/hat/2020-03-30 16:16:314.187707.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:314.303683.jpg
**********
217.0
218.0
成功保存人脸到./output/hat/2020-03-30 16:16:314.30608.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-

成功保存人脸到./output/2020-03-30 16:16:367.751068.jpg
**********
126.0
127.0
成功保存人脸到./output/hat/2020-03-30 16:16:367.752784.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得2个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:367.817952.jpg
**********
成功保存人脸到./output/2020-03-30 16:16:367.817952.jpg
**********
83.0
18.0
成功保存人脸到./output/hat/2020-03-30 16:16:367.819695.jpg
**********
成功保存人脸到./output/hat/2020-03-30 16:16:367.819695.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得3个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:367.898791.jpg
**********
131.0
129.0
成功保存人脸到./output/hat/2020-03-30 16:16:367.900202.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
********

rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得2个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:3810.723011.jpg
**********
成功保存人脸到./output/2020-03-30 16:16:3810.723011.jpg
**********
239.0
239.0
成功保存人脸到./output/hat/2020-03-30 16:16:3810.727232.jpg
**********
成功保存人脸到./output/hat/2020-03-30 16:16:3810.727232.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:3810.842905.jpg
**********
169.0
169.0
成功保存人脸到./output/hat/2020-03-30 16:16:3810.844823.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:3810.943307.jpg
**********
127.0
126.0
成功保存人脸到./output/hat/2020-03-30 16:16:3810.94486.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet

Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4014.05357.jpg
**********
141.0
140.0
成功保存人脸到./output/hat/2020-03-30 16:16:4014.054802.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得13个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4014.16198.jpg
**********
401.0
449.0
成功保存人脸到./output/hat/2020-03-30 16:16:4014.165869.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4014.256661.jpg
**********
263.0
263.0
成功保存人脸到./output/hat/2020-03-30 16:16:4014.259032.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得3个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********


Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得2个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4317.548339.jpg
**********
成功保存人脸到./output/2020-03-30 16:16:4317.548339.jpg
**********
281.0
282.0
成功保存人脸到./output/hat/2020-03-30 16:16:4317.553107.jpg
**********
成功保存人脸到./output/hat/2020-03-30 16:16:4317.553107.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4317.70611.jpg
**********
281.0
281.0
成功保存人脸到./output/hat/2020-03-30 16:16:4317.708992.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得3个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:16:4317.893782

...

Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4087.585107.jpg
**********
291.0
290.0
成功保存人脸到./output/hat/2020-03-30 16:17:4087.588974.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4087.706968.jpg
**********
131.0
106.0
成功保存人脸到./output/hat/2020-03-30 16:17:4087.708755.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4187.880725.jpg
**********
564.0
519.0
成功保存人脸到./output/hat/2020-03-30 16:17:4187.891362.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
*********

准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4492.324746.jpg
**********
275.0
259.0
成功保存人脸到./output/hat/2020-03-30 16:17:4492.32792.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得15个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4492.459514.jpg
**********
291.0
291.0
成功保存人脸到./output/hat/2020-03-30 16:17:4492.464025.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4492.59581.jpg
**********
115.0
133.0
成功保存人脸到./output/hat/2020-03-30 16:17:4492.597693.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过R

Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4796.586243.jpg
**********
106.0
140.0
成功保存人脸到./output/hat/2020-03-30 16:17:4796.588184.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4796.66527.jpg
**********
71.0
72.0
成功保存人脸到./output/hat/2020-03-30 16:17:4796.666535.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:4796.779688.jpg
**********
349.0
298.0
成功保存人脸到./output/hat/2020-03-30 16:17:4796.783298.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测

开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:49100.088983.jpg
**********
97.0
97.0
成功保存人脸到./output/hat/2020-03-30 16:17:49100.090251.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:50100.316569.jpg
**********
113.0
112.0
成功保存人脸到./output/hat/2020-03-30 16:17:50100.317964.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得12个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-

Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:53104.579051.jpg
**********
235.0
234.0
成功保存人脸到./output/hat/2020-03-30 16:17:53104.581665.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:53104.752418.jpg
**********
73.0
73.0
成功保存人脸到./output/hat/2020-03-30 16:17:53104.754607.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得11个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:17:53104.885767.jpg
**********
205.0


In [14]:
asdf

NameError: name 'asdf' is not defined

In [15]:
import glob
data_list = glob.glob("./student/*/*") # 1813张图片
i = 0
cost_time = 0
for path in data_list:
    
    img = cv2.imread(path)
    
    start = time.clock()
    _, _, ret = mtcnn.detect_face(img)
    end = time.clock()
    
    cost_time += ((end-start)*1000)
    
    if ret==1:
        
        i += 1
        
        
print("准确率:{}".format(i/len(data_list)))
print("平均耗时:{}".format(cost_time/len(data_list)))

Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:30107.9769.jpg
**********
153.0
192.0
成功保存人脸到./output/hat/2020-03-30 16:20:30107.978387.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:30108.038394.jpg
**********
153.0
178.0
成功保存人脸到./output/hat/2020-03-30 16:20:30108.039611.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:30108.099605.jpg
**********
169.0
216.0
成功保存人脸到./output/hat/2020-03-30 16:20:30108.101042.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得12个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
***

成功保存人脸到./output/hat/2020-03-30 16:20:32110.074158.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:32110.144576.jpg
**********
202.0
201.0
成功保存人脸到./output/hat/2020-03-30 16:20:32110.146408.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:32110.21705.jpg
**********
206.0
206.0
成功保存人脸到./output/hat/2020-03-30 16:20:32110.217985.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得12个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:32110.290394.jpg
**********
202.0
201.0
成功保存人脸到./output/hat/2020-03-30 16:20:32110.292381.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rn

开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得2个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:34112.263142.jpg
**********
成功保存人脸到./output/2020-03-30 16:20:34112.263142.jpg
**********
148.0
149.0
成功保存人脸到./output/hat/2020-03-30 16:20:34112.265887.jpg
**********
成功保存人脸到./output/hat/2020-03-30 16:20:34112.265887.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:34112.330112.jpg
**********
192.0
192.0
成功保存人脸到./output/hat/2020-03-30 16:20:34112.331676.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:34112.390205.jpg
**********
178.0
177.0
成功保存人脸到./output/hat/2020-03-30 16:20:34112.391431.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
******

Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:35114.206029.jpg
**********
137.0
153.0
成功保存人脸到./output/hat/2020-03-30 16:20:35114.207127.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:35114.277214.jpg
**********
153.0
163.0
成功保存人脸到./output/hat/2020-03-30 16:20:35114.278102.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:35114.347212.jpg
**********
144.0
163.0
成功保存人脸到./output/hat/2020-03-30 16:20:35114.348891.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得6个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
****

onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:37116.246884.jpg
**********
235.0
196.0
成功保存人脸到./output/hat/2020-03-30 16:20:37116.249185.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得2个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:37116.325691.jpg
**********
236.0
256.0
成功保存人脸到./output/hat/2020-03-30 16:20:37116.328475.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得14个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:37116.401097.jpg
**********
256.0
256.0
成功保存人脸到./output/hat/2020-03-30 16:20:37116.405166.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:20:37116.469918.jpg
**********
211.0
235.0
成功保存人脸到./output/hat/

...

Pnet网络处理完毕
**********
pnet一共获得12个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:12284.545948.jpg
**********
160.0
177.0
成功保存人脸到./output/hat/2020-03-30 16:23:12284.548321.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得13个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:12284.64892.jpg
**********
150.0
168.0
成功保存人脸到./output/hat/2020-03-30 16:23:12284.650522.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得13个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:12284.743994.jpg
**********
150.0
168.0
成功保存人脸到./output/hat/2020-03-30 16:23:12284.74687.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得13个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**

onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:15287.504322.jpg
**********
129.0
129.0
成功保存人脸到./output/hat/2020-03-30 16:23:15287.505502.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:15287.603881.jpg
**********
148.0
149.0
成功保存人脸到./output/hat/2020-03-30 16:23:15287.60532.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得7个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:15287.697593.jpg
**********
139.0
140.0
成功保存人脸到./output/hat/2020-03-30 16:23:15287.699611.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得11个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:15287.799445.jpg
**********
140.0
140.0
成功保存人脸到./output/hat/

Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:17290.490164.jpg
**********
160.0
143.0
成功保存人脸到./output/hat/2020-03-30 16:23:17290.492731.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得8个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:17290.581893.jpg
**********
144.0
173.0
成功保存人脸到./output/hat/2020-03-30 16:23:17290.584149.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得5个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:17290.669813.jpg
**********
179.0
203.0
成功保存人脸到./output/hat/2020-03-30 16:23:17290.672816.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
****

Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:23294.71332.jpg
**********
1028.0
1028.0
成功保存人脸到./output/hat/2020-03-30 16:23:23294.726709.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:23294.832473.jpg
**********
281.0
329.0
成功保存人脸到./output/hat/2020-03-30 16:23:23294.8347.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得4个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:24294.919021.jpg
**********
281.0
329.0
成功保存人脸到./output/hat/2020-03-30 16:23:24294.921591.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得9个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
*****

成功保存人脸到./output/2020-03-30 16:23:37301.644874.jpg
**********
177.0
178.0
成功保存人脸到./output/hat/2020-03-30 16:23:37301.647475.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得11个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得17个预测人脸框
**********
开始通过Rnet网络进行处理
**********
准备检测下一张图片
**********
Pnet网络处理完毕
**********
pnet一共获得10个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:37301.832003.jpg
**********
198.0
201.0
成功保存人脸到./output/hat/2020-03-30 16:23:37301.834373.jpg
**********
Pnet网络处理完毕
**********
pnet一共获得11个预测人脸框
**********
开始通过Rnet网络进行处理
**********
rnet一共获得1个预测人脸框
**********
开始通过Rnet网络进行处理
**********
onet一共获得1个预测人脸框
**********
成功保存人脸到./output/2020-03-30 16:23:37301.908166.jpg
**********
176.0
170.0
成功保存人脸到./output/hat/2020-03-30 16:23:37301.910482.jpg
**********
准确率:0.9553226696083839
平均耗时:102.69898069498132


In [ ]:
time.clock()

In [ ]:
time.clock()